In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from datetime import datetime

# sklearn
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics

# keras
import keras
from keras import models, optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras import Model ,models, layers, optimizers, regularizers
from keras.callbacks import ModelCheckpoint
import keras.backend.tensorflow_backend as K
import tensorflow as tf

import gc

In [ ]:
train_label = np.load('dataset/train_label.npy')
valid_label = np.load('dataset/valid_label.npy')
test_label = np.load('dataset/test_label.npy')

train_scaled = np.load('dataset/train_scaled.npy')
valid_scaled = np.load('dataset/valid_scaled.npy')
test_scaled = np.load('dataset/test_scaled.npy')

In [ ]:
timesteps = 30
# timesteps에 맞게 자르기
train_rest = -(train_scaled.shape[0] % timesteps)
valid_rest = -(valid_scaled.shape[0] % timesteps)
test_rest = -(test_scaled.shape[0] % timesteps)

train_scaled = train_scaled[:train_rest]
train_label = train_y[:train_rest]

valid_scaled = valid_scaled[:valid_rest]
valid_label = valid_label[:valid_rest]

test_scaled = test_scaled[:test_rest]
test_lable = test_label[:test_rest]


# 차원 변환
train_scaled_reshape = train_scaled.reshape(int(train_scaled.shape[0]/timesteps), timesteps, train_scaled.shape[1])
train_label_reshape = train_label.reshape(int(train_label.shape[0]/timesteps), timesteps, )

valid_scaled_reshape = valid_scaled.reshape(int(valid_scaled.shape[0]/timesteps), timesteps, valid_scaled.shape[1])
valid_label_reshape = valid_label.reshape(int(valid_label.shape[0]/timesteps), timesteps, )

test_scaled_reshape = test_scaled.reshape(int(test_scaled.shape[0]/timesteps), timesteps, test_scaled.shape[1])
test_label_reshape = test_label.reshape(int(test_label.shape[0]/timesteps), timesteps, )

In [ ]:
timesteps=30
unit=128
epoch=25
batch_size=100


# 모델생성
with tf.device('/gpu:0'):
    model = Sequential()
    model.add(LSTM(units=unit, input_shape=(timesteps, 78)))
    model.add(Dense(timesteps,activation='sigmoid'))
    model.compile(
                loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'],
                )
    model.summary()
    history = model.fit(train_scaled_reshape, train_label_reshape, epochs=epoch, batch_size=batch_size, 
                        validation_data=(test_scaled_reshape, test_label_reshape), verbose=1, shuffle=False)
    model.save('./model/lstm.h5')

In [ ]:
lstm = models.load_model('./model/lstm.h5')

predict_result = lstm.predict(valid_scaled_reshape)
result = predict_result.reshape(predict_result.shape[0]*predict_result.shape[1],1)
label = valid_label

best_f1 = 0
best_pr = 0
best_th = 0

for i in np.arange(0.1, 0.9, 0.01):
    result_cp = result.copy()
    
    result_cp[result>=i] = 1
    result_cp[result<i] = 0
    
    f1 = f1_score(label, result_cp)*100
    pr = 0
    if f1 > best_f1:
        pr = precision_score(label, result_cp)*100
        if pr > best_pr:
            best_th = i
            best_f1 = f1
            best_pr = pr
    
    del[[result_cp]]
print('\n\n')        
print('best f1 : ', best_f1)
print('best th : ', best_th)

In [ ]:
valid_preditions = model.predict(test_scaled_reshape)
result = valid_preditions.reshape(valid_preditions.shape[0]*valid_preditions.shape[1],1)
label = test_label

result_cp = result.copy()

result_cp[result>=best_th] = 1
result_cp[result<best_th] = 0

print("악성 분류 기준:", best_th)
print("accuracy_score :",accuracy_score(test_lable, result_cp)*100)
print("recall_score :",recall_score(test_lable, result_cp)*100)
print("precision_score :",precision_score(test_lable, result_cp)*100)
print("f1_score :",f1_score(test_lable, result_cp)*100)

del[[result_cp]]